In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTModel
from torch.nn.functional import softmax, one_hot
import pandas as pd 
import os
from tqdm import tqdm
from torchvision.transforms import transforms
import matplotlib.cm as cm
from PIL import Image
import numpy as np
import wandb

In [7]:
df = pd.read_csv("../this_studio/train.csv")

In [8]:
df.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [9]:
classes = df['expert_consensus'].unique()

In [10]:
mapping = {
    c:i for i, c in enumerate(classes)
}

In [11]:
num_classes = classes.shape[0]

In [12]:
cmap = cm.get_cmap("viridis")

/tmp/ipykernel_75110/4067785149.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("viridis")


In [13]:
class spectrogramDataset(Dataset):
    def __init__(self,csv_file,path="../this_studio/train_spectrograms/",transform=None):
        self.df = pd.read_csv(csv_file).iloc[:50,:]
        self.transform = transform
        self.path = path
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,idx):
        spec_id = df.iloc[idx,:]['spectrogram_id']
        label = df.iloc[idx,:]['expert_consensus']
        
        spectrogram_path = os.path.join(self.path,str(spec_id)+".parquet")
        # print(spectrogram_path)
        spectrogram = pd.read_parquet(spectrogram_path).drop('time',axis=1).values
        # print(spectrogram)
        spectrogram = Image.fromarray((cmap(spectrogram) * 255).astype(np.uint8))
        
        if self.transform:
            spectrogram = self.transform(spectrogram)[:3, :, :]
        
        
        return spectrogram,mapping[label]
        
        

In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
#     transforms.CenterCrop(224),  # Center crop to maintain aspect ratio
    transforms.ToTensor(),  # Convert to PyTorch tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406, 0.406], std=[0.229, 0.224, 0.225, 0.225])  # Normalize (optional)
])

In [15]:
dataset = spectrogramDataset("../this_studio/train.csv",transform=transform)

In [16]:
dataset.__len__()

50

In [17]:
# dataset.__getitem__(10000)

In [18]:
train_len = int(.8*len(dataset))
val_len = len(dataset)-train_len

In [19]:
val_len

10

In [20]:
train_set,val_set = torch.utils.data.random_split(dataset,[train_len,val_len])

In [22]:
train_loader = DataLoader(train_set,batch_size=5,shuffle=True, num_workers = 2, prefetch_factor=1)
val_loader = DataLoader(val_set, batch_size=5,shuffle=True,num_workers = 2, prefetch_factor = 1)

In [23]:
class ViTClassifier(torch.nn.Module):
    def __init__(self,num_classes=num_classes):
        super().__init__()
        self.vit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
        self.classifier = torch.nn.Linear(self.vit.config.hidden_size,num_classes)
        
    def forward(self,images):
        output = self.vit(images)
        output = self.classifier(output.last_hidden_state[:,0])
        output = softmax(output,dim=1)
        return output

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
model = ViTClassifier(num_classes=num_classes).to(device)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [26]:
loss_fn = torch.nn.KLDivLoss(reduction='batchmean')
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [27]:
args = {
    "lr":.001,
    "loss":"KLDivloss"
}

In [28]:
wandb.init(config=args)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jatinsingh. Use `wandb login --relogin` to force relogin


In [29]:
wandb.watch(model, log_freq=1)

[]

In [30]:
num_epochs = 5

#early stopping
patience = 5
early_stopping_counter = 0
best_validation_loss = float('inf')

#model checkpoint
checkpoint_path = "./Harmful-brain-activity-pytorch/models/checkpoints/ck_model.pth"

In [33]:
for epoch in range(num_epochs):
    pbar = tqdm(train_loader)
    
    for spectrograms, labels in pbar:
        labels_onehot = one_hot(labels,num_classes=num_classes).float().to(device)
        spectrograms = spectrograms.to(device)
        optimizer.zero_grad()
        
        outputs = model(spectrograms)
        loss = loss_fn(outputs.log(),labels_onehot)
        loss.backward()
        optimizer.step()
        pbar.set_description(f'train loss: {loss}')
    wandb.log({"train loss":loss})

    
        
    with torch.no_grad():
        correct = total=val_loss = 0
        pbar = tqdm(val_loader)
        
        for spectrograms,labels in pbar:
            labels = labels.to(device)
            labels_onehot = one_hot(labels,num_classes).float()
            spectrograms = spectrograms.to(device)
            outputs = model(spectrograms)
            
            _, predicted = torch.max(outputs.data,1)
            
            total+=labels.size(0)
            
            correct += (predicted==labels).sum().item()
            loss = loss_fn(outputs.log(),labels_onehot).item()
            val_loss+=loss
            pbar.set_description(f'val loss: {loss}')
        wandb.log({"val loss":loss})

        accuracy = 100 * correct/total
        
        kl_divergence = val_loss / len(val_loader)
        
        wandb.log({"accuracy":accuracy,"kl divergence":kl_divergence})
        
        print("Validation Accuracy: ", accuracy)
        print("KL divergence loss: ",kl_divergence )

        if best_validation_loss - val_loss < .01:
            best_validation_loss = val_loss
            early_stopping_counter = 0
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, checkpoint_path)
        else:
            early_stopping_counter+=1
        
        if early_stopping_counter >= patience:
            break; 
        
torch.save(model.state_dict(),'../this_studio/Harmful-brain-activity-pytorch/models/trained_model.pt')        

val loss: 1.6309314966201782: 100%|██████████| 2/2 [00:02<00:00,  1.17s/it]


Validation Accuracy:  20.0
KL divergence loss:  1.6829172372817993


val loss: 0.8028178215026855: 100%|██████████| 2/2 [00:02<00:00,  1.17s/it]


Validation Accuracy:  60.0
KL divergence loss:  1.1787062883377075


val loss: 1.3655271530151367: 100%|██████████| 2/2 [00:02<00:00,  1.31s/it]


Validation Accuracy:  60.0
KL divergence loss:  1.0078765749931335


val loss: 0.9612978100776672: 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]


Validation Accuracy:  70.0
KL divergence loss:  1.0199852287769318


val loss: 1.298303246498108: 100%|██████████| 2/2 [00:02<00:00,  1.41s/it] 


Validation Accuracy:  30.0
KL divergence loss:  1.1911728978157043
